In [ ]:
# Importing the Keras libraries and packages
import self as self
from keras.models import Sequential, load_model

"""There are two basic ways of initialising a neural network, either by a sequence of layers or as a graph"""
from keras.layers import Conv2D
"""Since we are working on images here, which a basically 2 Dimensional arrays, we’re using Convolution 2-D,
you may have to use Convolution 3-D while dealing with videos"""
from keras.layers import MaxPooling2D
"""There exist different types of pooling operations like Min Pooling, Mean Pooling, etc.
Here in MaxPooling we need the maximum value pixel from the respective region of interest"""
from keras.layers import Flatten
"""Flattening is the process of converting all the resultant 2 dimensional arrays into a single long continuous linear vector"""
from keras.layers import Dense
"""In order to perform the full connection of the neural network"""
from keras.preprocessing.image import ImageDataGenerator
"""In order to create synthetic data out of the same images by performing different type of operations on these images like flipping, rotating, blurring, etc"""
import numpy as np
from keras.preprocessing import image
"""Final predictions"""
import h5py
import math
import os
from scipy import ndimage





In [ ]:

############################################### THE SEQUENTIAL MODEL ###############################################
"""
STEP1: CONVOLUTION
We took the object which already has an idea of how our neural network is going to be(Sequential), then we added a convolution layer by using the “Conv2D” function.
The Conv2D function is taking 4 arguments, the first is the number of filters i.e 32 here, the second argument is the shape each filter is going to be i.e 3x3 here,
the third is the input shape and the type of image(RGB or Black and White)of each image i.e the input image our CNN is going to be taking is of a 64x64 resolution
and “3” stands for RGB, which is a colour img, the fourth argument is the activation function we want to use, here ‘relu’ stands for a rectifier function.
"""

classifier = Sequential()
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

"""
STEP2: POOLING
Now, we need to perform pooling operation on the resultant feature maps we get after the convolution operation is done on an image. The primary aim of a pooling
operation is to reduce the size of the images as much as possible, to reduce the total number of nodes for the upcoming layers.We start by taking our classifier
object and add the pooling layer. We take a 2x2 matrix we’ll have minimum pixel loss and get a precise region where the feature are located.
"""
classifier.add(MaxPooling2D(pool_size = (2, 2)))

"""
STEP3: FLATTERN
It’s time for us to now convert all the pooled images into a continuous vector through Flattening. Flattening is a very important step to understand. What we are
basically doing here is taking the 2-D array, i.e pooled image pixels and converting them to a one dimensional single vector.
"""
classifier.add(Flatten())

"""
STEP4: FULL CONNECTION
In this step we need to create a fully connected layer, and to this layer we are going to connect the set of nodes we got after the flattening step, these nodes
will act as an input layer to these fully-connected layers. As this layer will be present between the input layer and output layer, we can refer to it a hidden layer.
As you can see, Dense is the function to add a fully connected layer, ‘units’ is where we define the number of nodes that should be present in this hidden layer, these
units value will be always between the number of input nodes and the output nodes but the art of choosing the most optimal number of nodes can be achieved only through
experimental tries. Though it’s a common practice to use a power of 2. And the activation function will be a rectifier function.
"""
classifier.add(Dense(units = 50, activation = 'relu'))

"""
STEP5: DEFINE THE OUTPUT LAYER
Now it’s time to initialise our output layer, which should contain only one node, as it is binary classification. This single node will give us a binary output of either a Cat or Dog.
"""
classifier.add(Dense(units = 21871, activation = 'softmax')) #For binary classification:units = 4

############################################### COMPILATION ###############################################
"""
Optimizer parameter is to choose the stochastic gradient descent algorithm. Loss parameter is to choose the loss function.
Finally, the metrics parameter is to choose the performance metric.
"""
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
############################################### THE SEQUENTIAL MODEL ###############################################
"""
STEP1: CONVOLUTION
We took the object which already has an idea of how our neural network is going to be(Sequential), then we added a convolution layer by using the “Conv2D” function.
The Conv2D function is taking 4 arguments, the first is the number of filters i.e 32 here, the second argument is the shape each filter is going to be i.e 3x3 here,
the third is the input shape and the type of image(RGB or Black and White)of each image i.e the input image our CNN is going to be taking is of a 64x64 resolution
and “3” stands for RGB, which is a colour img, the fourth argument is the activation function we want to use, here ‘relu’ stands for a rectifier function.
"""



In [ ]:
###############################################   TRAINING   ###############################################
"""
In order to prevent Overfitting(when you get a great training accuracy and very poor test accuracy), the images of the dataset need to be preprocessed. We are going to do
this using keras.preprocessing library for doing the synthesising part as well as to prepare the training set as well as the test test set of images that are present in a properly
structured directories, where the directory’s name is take as the label of all the images present in it. The cleaning consists in flipping, rotating, blurring, etc.
"""
#######MAybe I should use rescale = 1./255 for get predictions
train_datagen = ImageDataGenerator(rescale = 1./255,shear_range = 0.2,zoom_range = 0.2,horizontal_flip = True)
validation_datagen = ImageDataGenerator(rescale = 1./255)
training_set = train_datagen.flow_from_directory('training_set',target_size = (64, 64),batch_size = 32,class_mode = 'categorical')
validation_set = validation_datagen.flow_from_directory('validation_set',target_size = (64, 64),batch_size = 32,class_mode = 'categorical')
test_set = validation_datagen.flow_from_directory('test_set',target_size = (64, 64),batch_size = 32,class_mode = 'categorical')

"""FITTING(‘steps_per_epoch’ holds the number of training images)"""
classifier.fit_generator(training_set,steps_per_epoch = 4000,epochs = 12,validation_data = validation_set,validation_steps = 1000)#8000 2000
"""Saving the weights into a file"""
classifier.save_weights('4000_1000_12.h5')

In [ ]:

###############################################  EVALUATION  ###############################################
scores=classifier.evaluate_generator(test_set)
print("[0] %s: %.2f%%" % (classifier.metrics_names[0], scores[0]*100))
print("[1] %s: %.2f%%" % (classifier.metrics_names[1], scores[1]*100))


In [ ]:

###############################################  PREDICTION  ###############################################
#model = load_model('model.h5')
'''test_image = image.load_img('prediction_set/a.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)'''

def softmax(z):
    z_exp = [math.exp(i) for i in z]
    sum_z_exp = sum(z_exp)
    return [i / sum_z_exp for i in z_exp]


prediction_datagen = ImageDataGenerator(rescale = 1./255)
prediction_set = prediction_datagen.flow_from_directory('prediction_set',target_size = (64, 64),batch_size = 32,class_mode = None)

'''results = classifier.predict_generator(self.test_generator)
predictions = np.argmax(results, axis=-1) #multiple categories
label_map = (predictions.class_indices)
label_map = dict((v,k) for k,v in label_map.items()) #flip k,v
predictions = [label_map[k] for k in predictions]'''

In [ ]:

results = classifier.predict_generator(prediction_set)
predictions = np.argmax(results, axis=-1) #multiple categories

print("Training dataset:")
print(training_set.classes)
print(training_set.class_indices)
print("Results")
print(softmax(results))
print(softmax(results[1]))
print(softmax(results[2]))
print(softmax(results[3]))
print(predictions)